In [6]:
from MK_MPII_loader import MPII

In [89]:
import imageio
import numpy as np
import skimage
import skimage.transform
from util import generate_heatmap
from vectormath import Vector3

In [135]:
mpii = MPII(batch_size=1, task='train')

In [143]:
img = mpii.batch_image_set()
img_idx = img[0][0]
annolist = mpii.mat['RELEASE'].annolist
imgpath = './data/images/' + annolist[img_idx].image.name
imageio.imwrite('original.jpg', skimage.img_as_float(skimage.io.imread(imgpath)))
batch_rgb, batch_heatmap, batch_keypoint, batch_activity, batch_threshold = mpii.get_minibatch(img)

C:\Users\Ethereum\Anaconda3\lib\site-packages\imageio\core\util.py:78: UserWarning: Lossy conversion from float64 to uint8, range [0, 1]
  dtype_str, out_type.__name__))
C:\Users\Ethereum\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [144]:
rgb = batch_rgb[0]
print(rgb.shape)
imageio.imwrite('rgb.jpg', rgb)

(256, 256, 3)


C:\Users\Ethereum\Anaconda3\lib\site-packages\imageio\core\util.py:78: UserWarning: Lossy conversion from float32 to uint8, range [0, 1]
  dtype_str, out_type.__name__))


In [145]:
heatmap = batch_heatmap[0]
print(heatmap.shape, type(heatmap))
print(np.min(heatmap), np.max(heatmap))

for y in range(64):
    for x in range(64):
        heatmap[y, x, 0] = max(heatmap[y, x, :])
imageio.imwrite('heatmap.jpg', heatmap[:, :, 0])

(64, 64, 16) <class 'numpy.ndarray'>
0.0 1.0


C:\Users\Ethereum\Anaconda3\lib\site-packages\imageio\core\util.py:78: UserWarning: Lossy conversion from float32 to uint8, range [0, 1]
  dtype_str, out_type.__name__))


In [146]:
print(batch_keypoint)

[[[27.901154 36.098846]
  [32.323593 39.981964]
  [37.39322  32.53932 ]
  [33.72583  29.195526]
  [27.469694 36.098846]
  [24.125898 39.011185]
  [35.559525 30.921356]
  [35.23593  24.449492]
  [32.419678 24.079473]
  [23.274761 22.87795 ]
  [38.579727 39.011185]
  [41.384205 34.157288]
  [37.93254  26.498915]
  [32.53932  22.292204]
  [34.049423 25.636   ]
  [31.24495  27.901154]]]


In [147]:
print(batch_activity)

[[618]]


In [148]:
print(batch_threshold)

[[25.6]]


In [149]:
colors = [
    Vector3(0x00, 0x00, 0x80), # Navy
    Vector3(0x00, 0x00, 0xFF), # Blue
    Vector3(0x00, 0xFF, 0x00), # Green
    Vector3(0xFF, 0xFF, 0x00), # Yellow
    Vector3(0xFF, 0x00, 0x00), # Red
    Vector3(0xFF, 0x00, 0x00), # Red
]

beautified = np.ndarray(shape=(64, 64, 3))
    
for y in range(64):
    for x in range(64):
        p = heatmap[y, x, 0] # in range [0.0, 1.0]
        index = int(p * 100) // 25 # 0 1 2 3 4
        
        t = (p - 0.25 * index)/ 0.25
        
        color = colors[index] * (1-t) + colors[index+1] * t
        beautified[y, x, :] = color

beautified = skimage.transform.resize(beautified, (256, 256))
imageio.imwrite('merged.jpg', rgb * 0.5 + beautified * 0.5 / 0xFF)

C:\Users\Ethereum\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\Ethereum\Anaconda3\lib\site-packages\imageio\core\util.py:78: UserWarning: Lossy conversion from float64 to uint8, range [0, 1]
  dtype_str, out_type.__name__))
